In [2]:
"""
Imports
"""
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from scipy.stats import ks_2samp
from scipy import stats
from matplotlib import pyplot
from sklearn import preprocessing
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
import sqlite3
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import plotly.plotly as py
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, precision_score
from sklearn import preprocessing
from ggplot import *
from bokeh.charts import TimeSeries
from bokeh.models import HoverTool
from bokeh.plotting import show
from bokeh.charts import Scatter, Histogram, output_file, show
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.charts import Bar, output_file, show
import bokeh.palettes as palettes
from bokeh.models import HoverTool, BoxSelectTool, Legend
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



In [3]:
HG002_crowd = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_250bp.csv')

In [4]:
HG002_crowd.head(3)

,chrom,start,end,sample,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,...,Ill250.GT,size,GTcons,GTconflict,GTsupp,CN0_prob,CN1_prob,CN2_prob,Svsize,Label
0,21,10842334,10842437,HG002,939.564516,27.786409,62,425.290323,66.341100,62,...,1,103,1,-1,2,0.02,0.04,0.94,103,2
1,10,42356979,42357254,HG002,878.684211,25.829507,19,413.000000,111.568247,19,...,1,275,1,-1,1,0.00,0.02,0.98,275,2
2,17,75845891,75846109,HG002,0.000000,0.000000,0,0.000000,0.000000,0,...,0,218,0,-1,2,0.00,0.09,0.91,218,2


In [5]:
### Drop irrelevant columns
# Note: will keep label column because these values will need to be stored in a dataframe
# Note: Label column is only kept for later storage in a dataframe
# Note: Missing values are also imputed in the Labels column using KNN
HG002_crowd.drop(['GTcons'], axis=1, inplace = True)
HG002_crowd.drop(['GTconflict'], axis=1, inplace = True)
HG002_crowd.drop(['GTsupp'], axis=1, inplace = True)
HG002_crowd.drop(['start'],axis=1, inplace = True)
HG002_crowd.drop(['end'],axis=1, inplace = True)
HG002_crowd.drop(['chrom'],axis=1, inplace = True)
HG002_crowd.drop(['sample'],axis=1, inplace = True)
HG002_crowd.drop(['CN0_prob'],axis=1, inplace = True)
HG002_crowd.drop(['CN1_prob'],axis=1, inplace = True)
HG002_crowd.drop(['CN2_prob'],axis=1, inplace = True)
HG002_crowd.drop(['Ill250.GT'],axis=1, inplace = True)

In [6]:
HG002_crowd.head(3)

,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,Ill250.alt_reason_insertSizeScore,Ill250.alt_reason_orientation,Ill250.amb_alnScore_mean,Ill250.amb_alnScore_std,...,Ill250.ref_alnScore_mean,Ill250.ref_alnScore_std,Ill250.ref_count,Ill250.ref_insertSize_mean,Ill250.ref_insertSize_std,Ill250.ref_reason_alignmentScore,Ill250.ref_reason_orientation,size,Svsize,Label
0,939.564516,27.786409,62,425.290323,66.341100,62,0,0,854.704663,88.695755,...,936.271429,31.350517,70,412.685714,82.282187,70,0,103,103,2
1,878.684211,25.829507,19,413.000000,111.568247,19,0,0,839.472471,70.977108,...,922.188679,34.409332,53,419.566038,82.120829,48,5,275,275,2
2,0.000000,0.000000,0,0.000000,0.000000,0,0,0,897.659483,116.510990,...,956.222222,22.503223,27,403.555556,86.352136,26,1,218,218,2


In [12]:
HG002_crowd.isnull().sum()

Ill250.alt_alnScore_mean                             0
Ill250.alt_alnScore_std                              0
Ill250.alt_count                                     0
Ill250.alt_insertSize_mean                           0
Ill250.alt_insertSize_std                            0
Ill250.alt_reason_alignmentScore                     0
Ill250.alt_reason_insertSizeScore                    0
Ill250.alt_reason_orientation                        0
Ill250.amb_alnScore_mean                             0
Ill250.amb_alnScore_std                              0
Ill250.amb_count                                     0
Ill250.amb_insertSize_mean                           0
Ill250.amb_insertSize_std                            0
Ill250.amb_reason_alignmentScore_alignmentScore      0
Ill250.amb_reason_alignmentScore_orientation         0
Ill250.amb_reason_flanking                           0
Ill250.amb_reason_insertSizeScore_alignmentScore     0
Ill250.amb_reason_insertSizeScore_insertSizeScore    0
Ill250.amb

In [13]:
X = HG002_crowd

In [8]:
X.head(3)

,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,Ill250.alt_reason_insertSizeScore,Ill250.alt_reason_orientation,Ill250.amb_alnScore_mean,Ill250.amb_alnScore_std,...,Ill250.ref_alnScore_mean,Ill250.ref_alnScore_std,Ill250.ref_count,Ill250.ref_insertSize_mean,Ill250.ref_insertSize_std,Ill250.ref_reason_alignmentScore,Ill250.ref_reason_orientation,size,Svsize,Label
0,939.564516,27.786409,62,425.290323,66.341100,62,0,0,854.704663,88.695755,...,936.271429,31.350517,70,412.685714,82.282187,70,0,103,103,2
1,878.684211,25.829507,19,413.000000,111.568247,19,0,0,839.472471,70.977108,...,922.188679,34.409332,53,419.566038,82.120829,48,5,275,275,2
2,0.000000,0.000000,0,0.000000,0.000000,0,0,0,897.659483,116.510990,...,956.222222,22.503223,27,403.555556,86.352136,26,1,218,218,2


** Impute missing values using KNN **

Used 3 of the nearest neighbors to impute missing values

In [9]:
# Convert dataframe to matrix
X=X.as_matrix()

#Imput missing values from three closest observations
X_imputed=KNN(k=3).complete(X)
X=pd.DataFrame(X_imputed)

ValueError: Input matrix is not missing any values

In [14]:
# Store Labels in a new 'Y' DataFrame
Y = pd.DataFrame()
Y['Label'] = X['Label']

In [15]:
pd.value_counts(Y['Label'].values, sort=False)

 0    552
 1    959
 2      3
-1      1
dtype: int64

In [17]:
X=X[['Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_insertSizeScore_insertSizeScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','size','Svsize']]

** Train Random Forest Classifier **

In [18]:
# Train Test Split
# Train on 70% of the data and test on 30%
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.7, random_state=0)

In [19]:
# Train Random Forest Classifier
model = RandomForestClassifier() 
model.fit(X_train, y_train)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

** Precision Score **
- Overall model performance
- Using 30% of original dataset (test set)
- Truth labels: CrowdVariant labels

In [20]:
model.predict(X_test)

array([0, 1, 1, ..., 0, 1, 0])

In [21]:
pred = model.predict(X_test)

In [22]:
precision_score(y_test, pred, average='micro') 

0.96229971724787933

***
Prediction 
***

Used svviz.HG002 data from first 5000 random selection

In [40]:
HG002_svviz = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/svviz/svviz_250bp.csv')

In [41]:
### Drop irrelevant columns
# Note: will keep label column because these values will need to be stored in a dataframe
# Note: Label column is only kept for later storage in a dataframe
# Note: Missing values are also imputed in the Labels column using KNN
HG002_svviz.drop(['GTcons'], axis=1, inplace = True)
HG002_svviz.drop(['GTconflict'], axis=1, inplace = True)
HG002_svviz.drop(['GTsupp'], axis=1, inplace = True)
HG002_svviz.drop(['start'],axis=1, inplace = True)
HG002_svviz.drop(['end'],axis=1, inplace = True)
HG002_svviz.drop(['chrom'],axis=1, inplace = True)
HG002_svviz.drop(['sample'],axis=1, inplace = True)
# HG002_svviz.drop(['Label'],axis=1, inplace = True)
HG002_svviz.drop(['GIAB_Crowd'],axis=1, inplace = True)
HG002_svviz.drop(['id'],axis=1, inplace = True)
HG002_svviz.drop(['type'],axis=1, inplace = True)
HG002_svviz.drop(['SVtype'],axis=1, inplace = True)

In [42]:
HG002_svviz.isnull().sum()

Size                                                0
Ill250.alt_alnScore_mean                            0
Ill250.alt_alnScore_std                             0
Ill250.alt_count                                    0
Ill250.alt_insertSize_mean                          0
Ill250.alt_insertSize_std                           0
Ill250.alt_reason_alignmentScore                    0
Ill250.alt_reason_insertSizeScore                   0
Ill250.alt_reason_orientation                       0
Ill250.amb_alnScore_mean                            0
Ill250.amb_alnScore_std                             0
Ill250.amb_count                                    0
Ill250.amb_insertSize_mean                          0
Ill250.amb_insertSize_std                           0
Ill250.amb_reason_alignmentScore_alignmentScore     0
Ill250.amb_reason_alignmentScore_orientation        0
Ill250.amb_reason_flanking                          0
Ill250.amb_reason_insertSizeScore_alignmentScore    0
Ill250.amb_reason_multimappi

In [43]:
X2 = HG002_svviz

In [44]:
X2=X2.as_matrix()

In [45]:
model.predict(X2)

ValueError: Number of features of the model must match the input. Model n_features is 31 and input n_features is 36 